## Frequency Shift Chirp Modulation: La modulación LoRa

LoRa emplea **Chirps** como base de cada símbolo modulado. Un **chirp** es una señal senoidal que aumenta o decrementa su frecuencia de forma lineal.


Cada símbolo $s(nT_s)$ es un numero real que se genera a partir de un vector binario $w(nT_s)$ de longitud igual al **Spreading Factor** (SF), el cual puede tomar valores entre 7 y 12.


- **Vector de bits**: $w(nT_s) = [w_0(nT_s), \dots, w_{\mathrm{SF}-1}(nT_s)]$, donde cada $w_h(nT_s) \in \{0, 1\}$.  

- **Combinaciones posibles**: este vector de $\mathrm{SF}$ bits admite $2^{\mathrm{SF}}$ símbolos distintos.



Para convertir el vector binario $w(nT_s)$ a un valor real $s(nT_s)$, usamos la siguiente  ecuación propuesta en el paper de Vangelista:

$$
s(nT_s) \;=\; \sum_{h=0}^{\mathrm{SF}-1} w_h(nT_s)\,\cdot\,2^{h}
$$

donde:

* $w_h(nT_s)$ es el bit en la posición $h$ del vector de bits $w(nT_s)$.

* Cada bit se multiplica por $2^h$ (su “peso”) y se suman todos los términos.


Podemos ver entonces que el símbolo $s(nT_s)$ puede tomar cualquiera de los valores enteros:

$$
s(nT_s) \;\in\; \{\,0, 1, 2, \dots, 2^{\mathrm{SF}} -1\}
$$




In [39]:
import numpy as np

def generate_random_uniform_bits(sf, nb_samples, ref):
    # Ajustamos nb_samples al múltiplo de sf más cercano por debajo
    nb_samples_adj = (nb_samples // sf) * sf    

    # Creamos un vector de numeros entre 0 y 1 con distribucion aleatoria
    z = np.random.uniform(size=nb_samples_adj)

    # Generamos los bits con una probabilidad a priori dada por ref
    bits = (z > ref).astype(np.uint8)

    return bits

def encoder(bits, sf):
    n_blocks = bits.size // sf
    # Restructuramos los bits en bloques de sf bits
    blocks = bits.reshape(n_blocks, sf)  

    # Generamos un vector con los pesos de cada bit
    weights = 2 ** np.arange(sf)[::-1]

    # Producto punto entre bits y pesos
    symbols = blocks.dot(weights)     

    return symbols

def decoder(symbols, sf):    
    symbols = np.asarray(symbols, dtype=np.uint64)
    # Preparamos un array vacío para los bits
    n_blocks = symbols.size
    bits = np.zeros((n_blocks, sf), dtype=np.uint8)

    # Creamos copia para no modificar el original
    vals = symbols.copy()
    
    # Empezamos por el bit menos significativo (LSB)
    # y vamos subiendo hasta el sf-1, guardándolo en la columna correspondiente.
    for i in range(sf):
        # Gracias a NumPy, (vals % 2) y (vals //= 2) se ejecutan en paralelo sobre
        # cada posición de vals, sin necesidad de bucle sobre símbolos.
        bits[:, sf - 1 - i] = (vals % 2).astype(np.uint8)   
        # Descartamos ese bit dividiendo por 2 (floor division)
        vals //= 2

    return bits

def BER(enviados, recibidos, numberOfsamples, sf):
    # Ajustamos al múltiplo de sf más cercano por debajo
    nb_samples_adj = (numberOfsamples // sf) * sf
     
    # Aplanamos la matriz de recibidos a 1D
    recibidos_flat = recibidos.flatten()[:nb_samples_adj]
    
    # Contamos diferencias y calculamos probabilidad de error
    errores = np.sum(enviados != recibidos_flat)
    pe = errores / nb_samples_adj
    
    print(f"\nLa Probabilidad de error en la decodificación es: {pe:.6f}")
    

# Ejemplo de uso:
SF = 9                     # puede ser cualquiera entre 7 y 12
nb_samples = 10000
bits_send = generate_random_uniform_bits(SF, nb_samples, 0.5)

print(bits_send[:9])           # los primeros 9 bits

bits_recv = decoder(encoder(bits_send, SF), SF)

print(bits_recv[0])           # los primeros 9 bits

BER(bits_send, bits_recv, nb_samples, SF)


[1 0 0 0 0 1 0 0 0]
[1 0 0 0 0 1 0 0 0]

La Probabilidad de error en la decodificación es: 0.000000
